# Zad 1.  
Zbadamy ewolucję układu na sieci kwadratowej o rozmiarze 100 ×100 dla początkowego losowego rozkładu dwóch opinii w układzie. Sporządzamy również wykres liczby opinii na ‘tak’ od czasu. Przyjmijmy sekwencyjny schemat aktualizacji opinii.  

In [1]:
# packages required
using Random, DataFrames, CSV, Tables

In [2]:
L = 100;
Random.seed!(2026);
snapshot_values = [0, 1164, 2328, 3492, 4656, 5820];

In [3]:
function save_grid_snapshot(grid, iteration)
    CSV.write("grid_snapshots/grid$iteration.csv",  Tables.table(grid), writeheader=false)
end;

In [4]:
function voter_model(; L=10, max_iter=1_000_000, verbose=true, savegrid=false)
    network = rand(0:1, (L, L)) # L x L randomized binary matrix {1: "yes", 0: "no")
    i = 0                       # iteration counter
    yesses = []                 # number of ones/yes answers container

    if (savegrid==true) && (i in snapshot_values)
        save_grid_snapshot(network, i)
    end 

    # model loop
    start_time = time()
    while (0 < sum(network) < L^2) && (i < max_iter)
    # sum(network) de facto means number of ones/yes answers,
    # having sum(network)==0 || sum(network)==L^2 means that model converged
        i += 1
        # foreach node in the network, copy opinion from one of its neighbout
        for x in 1:L
            for y in 1:L
                decision = rand(1:4)    # randomly pick neigbour to copy opinion from
                # why doesn't Julia natively support switch-case statement huh? -__-
                if decision == 1
                    # mod1() makes sure we dont go over the index range,
                    # insted we just go back to the start
                    network[x, y] = network[mod1(x - 1, L), y] # up
                elseif decision == 2
                    network[x, y] = network[mod1(x + 1, L), y] # down
                elseif decision == 3
                    network[x, y] = network[x, mod1(y - 1, L)] # left
                else
                    network[x, y] = network[x, mod1(y + 1, L)] # right
                end
            end
        end
        push!(yesses, sum(network)) # add number of ones / yes answers to the list 
        
        if (savegrid==true) && (i in snapshot_values)
            save_grid_snapshot(network, i)
        end
    end
    end_time = time()

    elapsed_time = end_time - start_time

    if verbose
        # end message using fancy ternary operator
        i == max_iter ? println("Model for L=$L did not converge, max_iter=$i reached") : println("Model for L=$L converged after $i iterations")
    end

    return (i, elapsed_time, yesses)

end;

In [5]:
(iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L, savegrid=true);

Model for L=100 converged after 5822 iterations


In [6]:
df = DataFrame(zad = 1,
               L = L,
               iter = iterations_passed,
               time = time_elapsed, 
               evolution = join(string.(ans_evolution), ";"));

# Zad 2.  
Powtarzamy dziesięciokrotnie symulacje mierząc średni czasy dojścia do konsensusu (dziesieć różnych punktów startowych). Wyznaczamy średnią z tych dziesięciu pomiarów i niepewność jej wyznaczenia.

In [7]:
for i in 1:10
    (iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L, verbose=false, savegrid=false);
    push!(df, (2, L, iterations_passed, time_elapsed, join(string.(ans_evolution), ";")))
end;

  # Zad 3.  
Sprawdźmy czy i jak średni czas dojścia do konsensusu (z dziesięciu realizacji) zależy od rozmiaru układu $L^2$? Przyjmijmy L= 10, 50, 100, 200.

In [8]:
for L in [10, 50, 200]    
    for i in 1:10
        (iterations_passed, time_elapsed, ans_evolution) = voter_model(L=L, verbose=false, savegrid=false);
        push!(df, (3, L, iterations_passed, time_elapsed, join(string.(ans_evolution), ";")))
    end
    println("10 x model with L=$L calculated")
end;

10 x model with L=10 calculated
10 x model with L=50 calculated
10 x model with L=200 calculated


In [9]:
df[1:5, 1:4] # skipped last column with iteration evolution 

5×4 DataFrame
 Row │ zad    L      iter   time     
     │ Int64  Int64  Int64  Float64  
─────┼───────────────────────────────
   1 │     1    100   5822  0.618108
   2 │     2    100  34980  3.68652
   3 │     2    100   6118  0.619034
   4 │     2    100   8024  0.799178
   5 │     2    100   4269  0.431265

In [10]:
CSV.write("results.csv", df);